In [1]:
import pandas as pd
import numpy as np
import h5py
from datetime import datetime, timedelta
import sys

from matplotlib.colors import TwoSlopeNorm, LogNorm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates

import geopandas as gpd

import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
import textwrap

In [2]:
folder_path = "../"
sys.path.append(folder_path)
sys.path.append("../mobility_function/")
from mobility_function import analysis as ma
from importlib import reload
import mobility_function.analysis as ma
import mobility_function.hurricane_plotting as mhp
ma = reload(ma)
mhp = reload(mhp)
print(dir(ma))

['MO', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'array_summary', 'cos_sim', 'datetime', 'get_diagonal', 'get_diagonal_prob', 'get_mondays', 'get_travelling_out', 'h5py', 'h5py_to_4d_array', 'np', 'pd', 'region_mobility', 'region_out_desitination', 'relativedelta', 'relave_diff_D', 'test', 'timedelta']


In [3]:
# Load the data
Ms_h_base = ma.h5py_to_4d_array(f'../data/mobility/M_raw_20240909.h5')
Ms_h0 = ma.h5py_to_4d_array(f'../data/mobility/M_raw_20240916.h5')
Ms_h = ma.h5py_to_4d_array(f'../data/mobility/M_raw_20240923.h5')
Ms_h1 = ma.h5py_to_4d_array(f'../data/mobility/M_raw_20240930.h5')
print('size of matrix:', Ms_h_base.shape)

size of matrix: (7, 17, 3144, 3144)


In [4]:
M_H = np.concatenate([Ms_h_base, Ms_h0, Ms_h, Ms_h1], axis=0)

In [5]:
### read back the selected counties based on the storm track
cutoff_mile = 50
hurricane = "helene"
with open("../results/{}/counties_geoid_cut_{}.txt".format(hurricane, cutoff_mile), "r") as f:
    county_list = [line.strip() for line in f]
county_list = [int(x) for x in county_list]
geo_idx = pd.read_csv('geoid_idx_names.csv')
selected_idx = geo_idx[geo_idx['GEOID'].isin(county_list)].county_idx.values
print('no of selected counties',len(selected_idx))
selected_names = geo_idx[geo_idx['GEOID'].isin(county_list)].NAME.values
### destination index
with open("../results/{}/dest_counties_geoid_{}.txt".format(hurricane, cutoff_mile), "r") as f:
    dst_county_list = [line.strip() for line in f]
dst_county_list = [int(x) for x in dst_county_list]
print('no of destination county',len(dst_county_list))
dst_idx = geo_idx[geo_idx['GEOID'].isin(dst_county_list)].county_idx.values
dst_names = geo_idx[geo_idx['GEOID'].isin(dst_county_list)].NAME.values

no of selected counties 270
no of destination county 230


In [7]:
M_regout_base = ma.region_out_desitination(Ms_h_base, selected_idx)
M_regout_0 = ma.region_out_desitination(Ms_h0,selected_idx)
M_regout = ma.region_out_desitination(Ms_h,selected_idx)
M_regout_1 = ma.region_out_desitination(Ms_h1,selected_idx)

In [8]:
M_regout_all = np.concatenate([M_regout_base,M_regout_0,M_regout,M_regout_1], axis=0)

In [9]:
gdf = gpd.read_file('../data/county_geo/tl_2023_us_county/tl_2023_us_county.shp')
gdf['GEOID'] = gdf['GEOID'].astype(int)
projected_crs = "EPSG:5070"  # USA Contiguous Albers Equal Area
geo_centers = gdf.to_crs(projected_crs)
### using the original crs 4326 for distance calculation as the hawaii and alaska are included
#### sort the df to match with the order of selected counties
geo_centers = geo_centers.merge(geo_idx, on=['GEOID', 'NAME'])
geo_centers.sort_values(by='county_idx', inplace=True)

set_rg = geo_centers[geo_centers['GEOID'].isin(county_list)]  
set_dst = geo_centers[geo_centers['GEOID'].isin(dst_county_list)]  # Next 310 elements
centroids_rg = set_rg.geometry.centroid
centroids_dst = set_dst.geometry.centroid

In [10]:
from geopy.distance import geodesic

# Convert centroids to WGS84 (EPSG:4326)
centroids_rg_4326 = centroids_rg.to_crs("EPSG:4326")
centroids_dst_4326 = centroids_dst.to_crs("EPSG:4326")
# Initialize the distance array
distance_array_km = np.zeros((len(centroids_rg), len(centroids_dst)))
# Calculate distances between all pairs of centroids
for i, center_rg in enumerate(centroids_rg_4326):
    for j, center_dst in enumerate(centroids_dst_4326):
        coords_rg = (center_rg.y, center_rg.x)  # (latitude, longitude)
        coords_dst = (center_dst.y, center_dst.x)  # (latitude, longitude)
        distance_array_km[i, j] = geodesic(coords_rg, coords_dst).kilometers

In [23]:
### sort properly!!!
# df_distance = pd.DataFrame(data=distance_array_km, index=selected_names, columns=dst_names)
# df_distance.to_csv('df_distance_helene.csv')

In [11]:
def count_counties(distance_array_km, threshold):
    check_matrix = distance_array_km<=threshold
    within_count = 0
    for des in range(len(dst_county_list)):
        within = np.sum(check_matrix[:,des]) >= 1
        if within:
            within_count += 1
    print('no of counties within {} km:'.format(threshold), within_count, within_count/len(dst_county_list))
    return within_count, within_count/len(dst_county_list)

In [12]:
_ , _ = count_counties(distance_array_km, 400)

no of counties within 400 km: 131 0.5695652173913044


In [13]:
def travel_dis_per_visit(M_region_dst, distance_array_km):
    no_selected_counties = M_region_dst.shape[0]
    no_dst_counties = M_region_dst.shape[1]
    travel_distance_ls = []
    for i in range(no_selected_counties):
        for j in range(no_dst_counties):
            dst_km_ij = distance_array_km[i, j]
            v_ij = int(round(M_region_dst[i, j])) ## round half to even
            travel_distance_ls.extend([dst_km_ij]*v_ij)
    weighted_avg = np.sum(distance_array_km*M_region_dst)/np.sum(M_region_dst)
    return travel_distance_ls, weighted_avg

In [14]:
print(M_regout.shape)

(7, 270, 3144)


In [15]:
M_region_out_hwk_sum = np.sum(M_regout,axis=0)
M_region_dst_hwk_sum = M_region_out_hwk_sum[:,dst_idx]
travel_distance_h_ls, w_dis_h = travel_dis_per_visit(M_region_dst_hwk_sum, distance_array_km)
print(cutoff_mile,'hurricane week', w_dis_h)
M_region_out_basewk_sum = np.sum(M_regout_base,axis=0)
M_region_dst_basewk_sum = M_region_out_basewk_sum[:,dst_idx]
travel_distance_base_ls, w_dis_base = travel_dis_per_visit(M_region_dst_basewk_sum, distance_array_km)
print('base week', w_dis_base)

50 hurricane week 420.2874714149899
base week 399.93267271801733


In [52]:
np.sum(M_region_out_hwk_sum)

np.float64(426003017.9951086)

In [51]:
np.sum(np.array(travel_distance_h_ls)<=400)

np.int64(60533899)

In [54]:
(60533899/426003017.9951086)*100

14.209734777206451

In [17]:
diff_m_rg_dst_sum = M_region_dst_hwk_sum - M_region_dst_basewk_sum

In [18]:
pos_change = [] 
neg_change = []
for i in range(len(selected_idx)):
    for j in range(len(dst_idx)):
        diff_ij = diff_m_rg_dst_sum[i,j]            
        if diff_ij > 0:
            pos_change.extend([distance_array_km[i,j]]*int(round(diff_ij)))
        else:
            neg_change.extend([distance_array_km[i,j]]*-int(round(diff_ij))
        )


In [19]:
bins = np.concatenate([np.linspace(0, 4500, 46), [float('inf')]])

counts_pos, bin_edges_pos = np.histogram(pos_change, bins=bins)
counts_neg, bin_edges_neg = np.histogram(neg_change, bins=bins)

wd_p = np.diff(bin_edges_pos)
wd_p[-1] = wd_p[0]
wd_n = np.diff(bin_edges_neg)
wd_n[-1] = wd_n[0]

pos_bin_centers = (bin_edges_pos[:-1] + bin_edges_pos[1:]) / 2
pos_bin_centers[-1] = bin_edges_pos[-2] + wd_p[0]

neg_bin_centers = (bin_edges_neg[:-1] + bin_edges_neg[1:]) / 2
neg_bin_centers[-1] = bin_edges_neg[-2] + wd_n[0]

In [20]:
np.savetxt('bins_{}_{}.txt'.format(hurricane,cutoff_mile), np.array([wd_n, neg_bin_centers, counts_neg, pos_bin_centers, counts_pos]))